# 1. 두 문장 단위 인퍼런스
**파일명 : inference.py**

In [ ]:
%%writefile inference.py

from torchtext import data
from engine.models.awkward import awkwardClassifier
from konlpy.tag import Okt

import torch
import argparse

path = "./"
#model_name = "model_gpu_nsp_cosine_v7.pth"
#model_name = "model_gpu_okt_layer2_seq64_v8.pth"
model_name = "model_gpu_okt_layer2_v9.pth"

use_eos=False
n_classes = 2

saved_data = torch.load(path + model_name, map_location = 'cuda:0' if torch.cuda.is_available() else 'cpu')

config = saved_data['config']
awkward_best = saved_data['awkward']
vocab1 = saved_data['vocab1']
vocab2 = saved_data['vocab2']
classes = saved_data['classes']

vocab1_size = len(vocab1)
vocab2_size = len(vocab2)

okt = Okt()

def infer(x1, x2):
    
    def define_field():

#         return (
#             data.Field(sequential = True,
#                                   use_vocab = True,
#                                   batch_first = True,
#                                   include_lengths = False,
#                                   eos_token='<EOS>' if use_eos else None,
#                                   pad_first=True
#                                   ),
#             data.Field(sequential = True,
#                                   use_vocab = True,
#                                   batch_first = True,
#                                   include_lengths = False,
#                                   eos_token='<EOS>' if use_eos else None,
#                                   pad_first=True
#                                   ),        
#             data.Field(sequential = False,
#                                  preprocessing = lambda x: int(x),
#                                      use_vocab = True,
#                                     init_token = None,
#                                      unk_token = None
#                                   )
#         )

        return (data.Field(sequential = True,
                              tokenize=okt.morphs, # data가 이미 tokenizing 되어 있음.
                              use_vocab = True,
                              batch_first = True,
                              include_lengths = False,
                              eos_token='<EOS>' if use_eos else None,
                              pad_first=True
                              ),
                 data.Field(sequential = True,
                              tokenize=okt.morphs,
                              use_vocab = True,
                              batch_first = True,
                              include_lengths = False,
                              eos_token='<EOS>' if use_eos else None,
                              pad_first=True
                              ),
        # sequential : If False,no tokenization is applied
        data.Field(sequential = False,
                             preprocessing = lambda x: int(x),
                                 use_vocab = True,
                                init_token = None,
                                 unk_token = None
                               )
    )
    

    text1_field, text2_field, label_field = define_field()
    
    text1_field.vocab = vocab1
    text2_field.vocab = vocab2 
    label_field.vocab = classes
    
    lines1 = []
    lines2 = []
    
    lines1 += [x1.strip().split(' ')]
    lines2 += [x2.strip().split(' ')]
    
    with torch.no_grad():
        x1 = text1_field.numericalize(
        text1_field.pad(lines1),
        device='cuda:0' if torch.cuda.is_available() else 'cpu',
        )

        x2 = text2_field.numericalize(
        text2_field.pad(lines2),
        device='cuda:0' if torch.cuda.is_available() else 'cpu',
        )

        model = awkwardClassifier(
                ntoken = vocab1_size,
                ntoken2 = vocab2_size,
                ninp = config.word_vec_size,
                n_classes=n_classes,
                nhead = config.nhead, 
                nhid = config.nhid, 
                nlayers = config.nlayers,
                cos = config.cos,
                dropout=config.dropout,
            )
        
        ensemble = []
        model.load_state_dict(awkward_best)
        ensemble += [model]
        
        if config.gpu_id >= 0:
            model.cuda(config.gpu_id)

        model.eval()
        
        y_hat = []
        
        def pad_to_maxseq_to_batch(batch, max_length, device=-1):
    
            if batch.size(1) >= max_length:
                batch = batch[:, :max_length]

            else:
                pad_num = max_length - batch.size(1)
                pad = torch.ones(batch.size(0), pad_num, device=device) # pad 값이 vocab index 1이라는 가정.
                batch = torch.cat([pad.long(), batch], dim=-1)

            return batch
        
        
    x1 = pad_to_maxseq_to_batch(x1, config.max_length, device='cuda:0' if torch.cuda.is_available() else 'cpu')
    x2 = pad_to_maxseq_to_batch(x2, config.max_length, device='cuda:0' if torch.cuda.is_available() else 'cpu')
    print(model(x1, x2)[0])    
    return torch.argmax(model(x1, x2)[0], dim=-1)

In [ ]:
# 1 : 비정상
# 0 : 정상
import inference

In [ ]:
line1 = "실제로 저는 조발표가 있는 수업을 좋아하지 않았고 특히 발표자 역할은 맡으려고 하지 않았습니다."
line2 = "하지만 프레젠테이션 능력은 어디를 가도 꼭 필요한 것임을 인지하게 되었고 이후에는 이를 개선하기 위해 노력했습니다."

In [ ]:
inference.infer(line1, line2)

# 2. 자기소개서 단건 Inferenece
**파일명 : inference_resume.py**

In [ ]:
%%writefile inference_resume.py

from torchtext import data
from engine.models.awkward import awkwardClassifier
from konlpy.tag import Okt

import torch
import kss
import argparse
import numpy as np

path = "./"
#model_name = "model_gpu_nsp_cosine_v7.pth"
model_name = "model_gpu_okt_layer2_seq64_v8.pth"
#model_name = "model_gpu_okt_layer2_v9.pth"

use_eos=False
n_classes = 2

saved_data = torch.load(path + model_name, map_location = 'cuda:0' if torch.cuda.is_available() else 'cpu')

config = saved_data['config']
awkward_best = saved_data['awkward']
vocab1 = saved_data['vocab1']
vocab2 = saved_data['vocab2']
classes = saved_data['classes']

vocab1_size = len(vocab1)
vocab2_size = len(vocab2)

okt = Okt()

def infer(resume):
    
    try:

        def define_field():

            return (data.Field(sequential = True,
                                  tokenize=okt.morphs, # data가 이미 tokenizing 되어 있음.
                                  use_vocab = True,
                                  batch_first = True,
                                  include_lengths = False,
                                  eos_token='<EOS>' if use_eos else None,
                                  pad_first=True
                                  ),
                     data.Field(sequential = True,
                                  tokenize=okt.morphs,
                                  use_vocab = True,
                                  batch_first = True,
                                  include_lengths = False,
                                  eos_token='<EOS>' if use_eos else None,
                                  pad_first=True
                                  ),
            # sequential : If False,no tokenization is applied
            data.Field(sequential = False,
                                 preprocessing = lambda x: int(x),
                                     use_vocab = True,
                                    init_token = None,
                                     unk_token = None
                                   )
        )

        text1_field, text2_field, label_field = define_field()

        text1_field.vocab = vocab1
        text2_field.vocab = vocab2 
        label_field.vocab = classes

        lines = kss.split_sentences(resume)

        line_len = len(lines)

        first_resume  = lines[:-1]
        second_resume = lines[1:]

        lines1 = []
        lines2 = []

        for i in first_resume:
            lines1 += [i.strip().split(' ')]

        for i in second_resume:
            lines2 += [i.strip().split(' ')]

        with torch.no_grad():
            x1 = text1_field.numericalize(
            text1_field.pad(lines1),
            device='cuda:0' if torch.cuda.is_available() else 'cpu',
            )

            x2 = text2_field.numericalize(
            text2_field.pad(lines2),
            device='cuda:0' if torch.cuda.is_available() else 'cpu',
            )

            model = awkwardClassifier(
                    ntoken = vocab1_size,
                    ntoken2 = vocab2_size,
                    ninp = config.word_vec_size,
                    n_classes=n_classes,
                    nhead = config.nhead, 
                    nhid = config.nhid, 
                    nlayers = config.nlayers,
                    cos = config.cos,
                    dropout=config.dropout,
                )

            ensemble = []
            model.load_state_dict(awkward_best)
            ensemble += [model]

            if config.gpu_id >= 0:
                model.cuda(config.gpu_id)

            model.eval()

            y_hat = []

            def pad_to_maxseq_to_batch(batch, max_length, device=-1):

                if batch.size(1) >= max_length:
                    batch = batch[:, :max_length]

                else:
                    pad_num = max_length - batch.size(1)
                    pad = torch.ones(batch.size(0), pad_num, device=device) # pad 값이 vocab index 1이라는 가정.
                    batch = torch.cat([pad.long(), batch], dim=-1)

                return batch

        x1 = pad_to_maxseq_to_batch(x1, config.max_length, device='cuda:0' if torch.cuda.is_available() else 'cpu')
        x2 = pad_to_maxseq_to_batch(x2, config.max_length, device='cuda:0' if torch.cuda.is_available() else 'cpu')
        
        result = torch.argmin(model(x1, x2), dim=-1)
        result_array = result.to(torch.device("cpu")).numpy()
        
        result_sentence = []
        
        for sen, result in zip(second_resume, result_array):
            if result == 0: # 이상문장일 경우
                result_sentence.append(sen)

    except Exception as e: 
        print("ERROR : ",e)
    
    return result_sentence

In [1]:
import inference_resume

In [8]:
# 아래 자기소개서는 기본적인 전처리가 된 상태여야 한다.
resume = "삼성전자 인턴 당시 저는 인턴 총무를 맡았습니다. 코로나 언제 끝나지. 총무를 맡았던 당시 잦은 인턴 회식으로 회비가 생각보다 많이 지출 되었습니다. 그래서 일부에서는 총무가 너무 헤프게 돈을 관리한다고 지적이 나왔었습니다. 그래서 저는 영수증과 앞으로의 계획을 문서화하여 논란이 있었던 부분을 해결하고자 하였습니다.먼저 투명한 예산공개를 통해 동기 인턴들에게 신뢰를 주었습니다. 그리고 불만을 말했던 동기들에게 먼저 다가가 사실관계를 설명 후 각자의 입장에 대해 이해하도록 노력하였습니다. 그리고 마지막으로 앞으로의 계획을 말하여 동기 전체의 이해와 스케줄관리를 하였습니다.저는 이런 갈등관계에서 소통이라는 해결방법을 선택하여, 갈등관계를 해결하고자 노력하였습니다. 이러한 방법은 서로의 입장을 이해 할 수 있었고, 다른 조직생활과 현재 근무지에서도 업무를 처리할 때에도 트러블이 나지 않고 좋은 관계를 유지중입니다.롯데 입사 후에도 상대방의 생각을 이해하고 소통을 통하여 좋은 관계를 유지하여 조직적 성과를 낼 수 있도록 노력하겠습니다."

In [9]:
result = inference_resume.infer(resume)

torch.Size([10, 64])


In [10]:
result

['코로나 언제 끝나지.',
 '먼저 투명한 예산공개를 통해 동기 인턴들에게 신뢰를 주었습니다.',
 '그리고 불만을 말했던 동기들에게 먼저 다가가 사실관계를 설명 후 각자의 입장에 대해 이해하도록 노력하였습니다.',
 '이러한 방법은 서로의 입장을 이해 할 수 있었고, 다른 조직생활과 현재 근무지에서도 업무를 처리할 때에도 트러블이 나지 않고 좋은 관계를 유지중입니다.']